In [1]:
import nilearn as nil
from nilearn import maskers, datasets
import json
import pickle

In [2]:
atlas = datasets.fetch_atlas_aal()
# Loading atlas image stored in 'maps'
atlas_filename = atlas['maps']

In [3]:
with open("./datasets.json", "r") as f:
    data = json.load(f)

In [4]:
bold = data["sub-292"]["ses-preop"]["run-01"]["bold"]
mask = data["sub-292"]["ses-preop"]["run-01"]["mask"]
confounds = data["sub-292"]["ses-preop"]["run-01"]["confounds"]

In [ ]:
for sub, ses in data.items():
    for op, runs in ses.items():
        for run, items in runs.items():
            masker = maskers.NiftiLabelsMasker(labels_img=atlas_filename, mask_img=items["mask"], standardize=True, t_r=items["TR"],
                                    memory='nilearn_cache', memory_level=1, verbose=5)
            items["time_series"] = masker.fit_transform(items["bold"])
print(data["sub-292"]["ses-preop"]["run-01"]["time_series"].shape)

In [6]:
with open("time_series.pkl", "wb") as f:
    pickle.dump(data, f)

In [10]:
with open("time_series.pkl", "rb") as f:
    data = pickle.load(f)